In [5]:
import cv2
import numpy as np
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
import json
import time
import os

# Cargar el mapeo de clases
with open('class_indices.json', 'r') as f:
    class_indices = json.load(f)

# Cargar el modelo TFLite
interpreter = tf.lite.Interpreter(model_path="modelo_vggface_senet50.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Configurar MTCNN para la detección de caras
detector = MTCNN()

# Función para preprocesar las imágenes
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = image.astype('float32')
    image = np.expand_dims(image, axis=0)
    return image

# Función para obtener la temperatura de la TPU (depende del entorno específico)
def get_tpu_temperature():
    try:
        with open('/sys/class/thermal/thermal_zone0/temp', 'r') as f:
            temp = f.read()
        return float(temp) / 1000  # Convertir de miligrados a grados Celsius
    except FileNotFoundError:
        print("TPU temperature file not found. Ensure the correct environment.")
        return None

# Función para obtener la memoria utilizada del sistema
def get_memory_used():
    try:
        with open('/proc/meminfo', 'r') as f:
            mem_info = f.readlines()
        
        # Leer la memoria total y libre
        mem_total = float(mem_info[0].split()[1])
        mem_free = float(mem_info[1].split()[1])
        
        # Calcular la memoria utilizada en MB
        mem_used = (mem_total - mem_free) / 1024  # Convertir de KB a MB
        return mem_used
    except FileNotFoundError:
        print("Memory info file not found. Ensure the correct environment.")
        return None

# Captura de la cámara en tiempo real
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Inicializar acumuladores para promedios
temp_accum = 0
mem_accum = 0
time_accum = 0
iterations = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Iniciar el temporizador
    start_time = time.time()

    # Obtener memoria utilizada del sistema
    mem_used = get_memory_used()

    # Obtener la temperatura de la TPU
    tpu_temp = get_tpu_temperature()

    # Detectar caras en la imagen
    faces = detector.detect_faces(frame)
    
    for face in faces:
        x, y, width, height = face['box']
        x1, y1, x2, y2 = x, y, x + width, y + height
        
        # Extraer la cara y preprocesar
        face_region = frame[y1:y2, x1:x2]
        face_region_preprocessed = preprocess_image(face_region)
        
        # Preparar la entrada para el modelo
        interpreter.set_tensor(input_details[0]['index'], face_region_preprocessed)
        
        # Ejecutar la inferencia
        interpreter.invoke()
        
        # Obtener el resultado de la inferencia
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predicted_class = np.argmax(output_data, axis=1)
        class_probability = output_data[0][predicted_class[0]]

        # Obtener el nombre de la clase
        class_name = class_indices[str(predicted_class[0])]
        
        # Mostrar resultados
        label = f"Class: {class_name}, Prob: {class_probability:.2f}"
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Calcular tiempo de inferencia
    elapsed_time = time.time() - start_time

    # Actualizar acumuladores
    iterations += 1
    if tpu_temp is not None:
        temp_accum += tpu_temp
    if mem_used is not None:
        mem_accum += mem_used
    time_accum += elapsed_time

    # Calcular promedios
    avg_temp = temp_accum / iterations if iterations > 0 else 0
    avg_mem = mem_accum / iterations if iterations > 0 else 0
    avg_time = time_accum / iterations if iterations > 0 else 0

    print(f"Avg TPU Temp: {avg_temp:.2f}°C, Avg Mem Used: {avg_mem:.2f} MB, Avg Infer Time: {avg_time:.4f} sec")
    
    cv2.imshow('Real-time Face Detection', frame)
    
    # Salir del bucle al presionar 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


Memory info file not found. Ensure the correct environment.
TPU temperature file not found. Ensure the correct environment.
1/1 [==============================] - 0s 8ms/step
Avg TPU Temp: 0.00°C, Avg Mem Used: 0.00 MB, Avg Infer Time: 0.6459 sec
Memory info file not found. Ensure the correct environment.
TPU temperature file not found. Ensure the correct environment.
1/1 [==============================] - 0s 50ms/step
Avg TPU Temp: 0.00°C, Avg Mem Used: 0.00 MB, Avg Infer Time: 0.8555 sec
Memory info file not found. Ensure the correct environment.
TPU temperature file not found. Ensure the correct environment.
1/1 [==============================] - 0s 10ms/step
Avg TPU Temp: 0.00°C, Avg Mem Used: 0.00 MB, Avg Infer Time: 0.7717 sec
Memory info file not found. Ensure the correct environment.
TPU temperature file not found. Ensure the correct environment.
13/13 [==============================] - 0s 3ms/step
Avg TPU Temp: 0.00°C, Avg Mem Used: 0.00 MB, Avg Infer Time: 0.7099 sec
Memory i

/Users/hector/miniconda3/envs/vgg_tfg/lib/python3.8/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
/Users/hector/miniconda3/envs/vgg_tfg/lib/python3.8/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
/Users/hector/miniconda3/envs/vgg_tfg/lib/python3.8/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
/Users/hector/miniconda3/envs/vgg_tfg/lib/python3.8/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 